In [1]:
import pandas as pd
import json
import re
import os
import subprocess
import bibtexparser
from tqdm import tqdm
import re
from flair.data import Sentence
from flair.nn import Classifier
from difflib import SequenceMatcher


/Users/quenzer/Desktop/Data Science/3. Semester 2023 WS/Data Analysis Project/A-blessing-or-a-curse-Analysis-of-scholarly-citations/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/quenzer/Desktop/Data Science/3. Semester 2023 WS/Data Analysis Project/A-blessing-or-a-curse-Analysis-of-scholarly-citations/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/quenzer/Desktop/Data Science/3. Semester 2023 WS/Data Analysis Project/A-blessing-or-a-curse-Analysis-of-scholarly-citations/.venv/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node

In [2]:
#TODO: write a function that normalizes the names of the authors from our annotations
# to fit the format generated by anystyle.io check below for examples and look for the pattern

In [3]:
def file_finder(file_str: str) -> str:
    """
    This function takes a file name and returns the path to the file in the all_data_articles.
    """
    title_doi = "../data/titles_doi.csv"
    folder_path = "../all_data_articles"
    
    #extract the doi from the file name
    doi = file_str.split("_")[-1].split(".")[0]

    # find the row in the csv file where the doi column ends with the doi
    df = pd.read_csv(title_doi)
    doi_row = df[df["DOI"].str.endswith(doi)]

    # extract the title from the row
    title_json = doi_row["Title"].values[0].replace(" ", "_") + ".json"

    for filename in os.listdir(folder_path):
        if filename.endswith(".json") and filename.startswith(title_json[:int(len(title_json)/3)]):
            return filename


In [4]:
def load_annotations(file_str: str) -> pd.DataFrame:
    """
    This function takes a file name and returns the annotations from the file.
    And also replaces missing values with None.
    """
    folder_path = "../data/annotated"

    file_path = os.path.join(folder_path, file_str)
    df = pd.read_excel(file_path)

    # replace missing values with None
    df = df.where(pd.notnull(df), None)

    # replace values marked with nan with None
    df = df.replace("nan", None)
    
    return df

In [5]:
def format_author_name(name):
    # TODO: probably need to handle more cases
    if name is None:
        return None
    if ' and ' in name:
        # Handle multiple authors
        authors = name.split(' and ')
        formatted_authors = [format_author_name(author) for author in authors]
        return ' and '.join(formatted_authors)
    else:
        parts = name.split()
        # Handle case where there is a middle initial
        if len(parts) == 3:
            return f"{parts[1]}, {parts[0]} {parts[2]}"
        # Handle case where there is no middle initial
        elif len(parts) == 2:
            return f"{parts[1]}, {parts[0]}"
        else:
            return name

In [6]:
def df_to_triplets(df: pd.DataFrame, format_author = True) -> set:
    """
    This function takes a dataframe and returns a set of triplets.
    """
    triplets = set()
    for i in range(len(df)):
        if format_author:
            triplet = (df.iloc[i]["Footnote"], format_author_name(df.iloc[i]["Authors"]), df.iloc[i]["Title"])
        else:
            triplet = (df.iloc[i]["Footnote"], df.iloc[i]["Authors"], df.iloc[i]["Title"])
        triplets.add(triplet)
    return triplets

In [7]:
def dict_to_triplets(extraction: dict) -> set:
    """
    Converts a dictionary of footnotes to a set of triplets
    """
    triplets = set()

    for number, references in extraction.items():
        for reference in references:
            author = reference[0]
            title = reference[1]

            if author == "":
                author = None

            if title == "":
                title = None
            
            triplets.add((int(number), author, title))

    return triplets

In [8]:
def information_extraction(file_path: str) -> set:
    """
    This function takes a file path and returns a set of triplets.
    """
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    extraction = {}

    prev_footnote = None

    for number, footnote in tqdm(article["footnotes"].items()):

        # If the footnote is ibid, use the previous footnote
        if footnote.startswith("Ibid"):
            footnote = prev_footnote

        # Store the footnote for the next iteration
        prev_footnote = footnote
        
        references = re.split(';',footnote)
        
        author_title_list = []

        for reference in references:

            command = ['ruby', 'anystyle.rb', str(reference)]
            bibtex = subprocess.run(command, stdout=subprocess.PIPE, text=True).stdout
            parsed_bibtex = bibtexparser.loads(bibtex).entries
            if parsed_bibtex:
                parsed_bibtex = parsed_bibtex[0]
            else:
                #print(f"No valid BibTeX entry found in: {bibtex}, set to empty dict")
                parsed_bibtex = {}

            #print(parsed_bibtex)

            # Extract title, prioritizing 'booktitle' if both 'title' and 'booktitle' are present
            title = parsed_bibtex.get('booktitle', parsed_bibtex.get('title', None))

            # Extract author information
            author = parsed_bibtex.get('author', None)

            # Append author and title pair to the list
            author_title_list.append([author, title])

        # Store the list in the extraction dictionary with the footnote number as the key
        extraction[number] = author_title_list

    return dict_to_triplets(extraction)

In [9]:
def calculate_scores(triplets, extractions):
    TP = len(triplets & extractions)  # Intersection of triplets and extractions
    FP = len(extractions - triplets)  # Elements in extractions but not in triplets
    FN = len(triplets - extractions)  # Elements in triplets but not in extractions

    recall = TP / (TP + FN) if TP + FN != 0 else 0
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    return recall, precision, f_score

In [10]:
# TODO: add some how to check for similarity between the authors and titles from the two sets. if for footnote number x 
# the authors and titles are very similar then we can assume that the extraction is correct.

def calculate_similarity(str1, str2):
    return SequenceMatcher(None, str1, str2).ratio()

def evaluate_extraction(set1, set2, threshold=0.95):
    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for triplet1 in set1:
        footnote1, author1, title1 = triplet1
        author1 = author1 if author1 is not None else ""
        title1 = title1 if title1 is not None else ""
        concat_str1 = str(author1) + " " + str(title1)
        found_match = False

        for triplet2 in set2:
            footnote2, author2, title2 = triplet2
            author2 = author2 if author2 is not None else ""
            title2 = title2 if title2 is not None else ""
            concat_str2 = str(author2) + " " + str(title2)

            # Check for footnote number and similarity
            if footnote1 == footnote2 and calculate_similarity(concat_str1, concat_str2) >= threshold:
                found_match = True
                break

        if found_match:
            true_positives += 1
        else:
            false_negatives += 1

    false_positives = len(set2) - true_positives

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f_score

In [11]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=True)
    extraction = information_extraction(title_json)
    recall, precision, f_score = evaluate_extraction(triplets, extraction)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")
    #break

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [01:28<00:00,  1.67it/s]




Recall: 0.5248618784530387
Precision: 0.4896907216494845
F-Score: 0.5066666666666667
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [01:24<00:00,  1.46it/s]




Recall: 0.29213483146067415
Precision: 0.2653061224489796
F-Score: 0.27807486631016043
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.4861111111111111
Precision: 0.4838709677419355
F-Score: 0.48498845265588914
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx


 21%|██        | 24/115 [00:28<01:38,  1.08s/it]Entry type thesis not standard. Not considered.
Entry type thesis not standard. Not considered.
100%|██████████| 115/115 [01:53<00:00,  1.01it/s]




Recall: 0.4143646408839779
Precision: 0.373134328358209
F-Score: 0.39267015706806285
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [01:16<00:00,  1.31it/s]



Recall: 0.29838709677419356
Precision: 0.1574468085106383
F-Score: 0.20612813370473537
--------------------------------------------------




In [12]:
# TODO: implement a simple approach with a just regrex to extract the author and title, and a simple split with ";"

def extract_citations(file_path: str) -> set:
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    
    citations = set()
    prev_footnote = None

    for footnote_number, footnote_text in tqdm(article["footnotes"].items()):
        # If the footnote is ibid, use the previous footnote
        if footnote_text.startswith("Ibid"):
            footnote_text = prev_footnote
        
        prev_footnote = footnote_text

        # Split the footnote into individual citations
        individual_citations = re.split(";", footnote_text)
        
        for citation_text in individual_citations:
            # Regular expression to extract authors and titles
            # TODO: try a better pattern
            pattern = re.compile(r'^(.+?),\s+(.+?)[,|(]')

            match = pattern.match(citation_text)
            
            if match:
                author = match.group(1)
                title = match.group(2)
                citations.add((int(footnote_number), author, title))


    return citations

 

In [13]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = extract_citations(title_json)
    recall, precision, f_score = evaluate_extraction(triplets, extraction)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")
    #break

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [00:00<00:00, 52020.20it/s]



Recall: 0.4277456647398844
Precision: 0.38144329896907214
F-Score: 0.4032697547683924
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [00:00<00:00, 162478.51it/s]




Recall: 0.3670886075949367
Precision: 0.29591836734693877
F-Score: 0.3276836158192091
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.5401069518716578
Precision: 0.46543778801843316
F-Score: 0.5
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx


100%|██████████| 115/115 [00:00<00:00, 321992.63it/s]




Recall: 0.32941176470588235
Precision: 0.27860696517412936
F-Score: 0.3018867924528302
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:00<00:00, 255127.98it/s]




Recall: 0.37719298245614036
Precision: 0.1829787234042553
F-Score: 0.2464183381088825
--------------------------------------------------




In [14]:
# TODO: write a function that extracts the author and title from the footnote using a tagger (i.e. flair)

def tagger_information_extraction(file_path: str, tagger) -> set:
    path = "../all_data_articles"
    file_path = os.path.join(path, file_path)
    article = json.load(open(file_path, "r"))
    
    citations = set()
    prev_footnote = None

    for footnote_number, footnote_text in tqdm(article["footnotes"].items()):
        # If the footnote is ibid, use the previous footnote
        if footnote_text.startswith("Ibid"):
            footnote_text = prev_footnote
        
        prev_footnote = footnote_text

        # Split the footnote into individual citations
        individual_citations = re.split(";", footnote_text)

        for citation_text in individual_citations:
            
            author = None

            sentence = Sentence(citation_text)
            tagger.predict(sentence)
            for span in sentence.get_spans('ner'):
                if span.tag == "PERSON" or span.tag == "ORG":
                    if author is None:
                        author = span.text
                    else:
                        author += ("and " + span.text)
                if span.tag == "WORK_OF_ART":
                    citations.add((int(footnote_number), author, span.text))
                    author = None

                
    return citations


In [15]:
path_annotations = "../data/annotated"
# load the NER tagger
tagger = Classifier.load('ner-ontonotes-large')

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    df = load_annotations(filename)
    triplets = df_to_triplets(df, format_author=False)
    extraction = tagger_information_extraction(title_json, tagger=tagger)
    recall, precision, f_score = evaluate_extraction(triplets, extraction, threshold=0.97)
    print("\n")
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 50)
    print("\n")

2023-12-18 14:13:50,322 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY
Labels - https___doi.org_10.1093_ehr_cew052.xlsx


100%|██████████| 148/148 [00:44<00:00,  3.36it/s]




Recall: 0.44651162790697674
Precision: 0.4948453608247423
F-Score: 0.46943765281173594
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead107.xlsx


100%|██████████| 124/124 [00:46<00:00,  2.68it/s]




Recall: 0.4039408866995074
Precision: 0.41836734693877553
F-Score: 0.4110275689223058
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead080.xlsx


...



Recall: 0.43915343915343913
Precision: 0.3824884792626728
F-Score: 0.4088669950738917
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_cead103.xlsx


100%|██████████| 115/115 [00:48<00:00,  2.39it/s]




Recall: 0.3076923076923077
Precision: 0.27860696517412936
F-Score: 0.2924281984334204
--------------------------------------------------


Labels - https___doi.org_10.1093_ehr_ceab280.xlsx


100%|██████████| 100/100 [00:33<00:00,  2.97it/s]



Recall: 0.36879432624113473
Precision: 0.22127659574468084
F-Score: 0.2765957446808511
--------------------------------------------------




In [16]:
def extract_triplets(triplets):
    return list(filter(lambda triplet: 10 < triplet[0] < 20, triplets))

In [17]:
extract_triplets(extraction)

[(12, None, '–1461'),
 (15, None, 'Bulletin of the Institute of Historical Research'),
 (15,
  'William Clerionetand Richard of Gloucesterand Katherineand A.J. Pollard',
  'Lord Fitzhugh’s Rising in 1470’'),
 (12,
  'R.A. Griffiths',
  'The Reign of King Henry VI: The Exercise of Royal Authority'),
 (11, 'S. Federico', 'The Imaginary Society: Women in 1381’'),
 (13, None, '1300–1348'),
 (13, 'B. Hanawalt', 'Crime and Conflict in English Communities'),
 (17, 'Warwickand Pollard', '‘Lord Fitzhugh’s Rising’'),
 (19,
  'Richard Salkeld',
  'Calendar of the Patent Rolls Preserved in the Public Record Office')]

In [18]:
extract_triplets(triplets)

[(11.0, 'S. Federico', 'The Imaginary Society: Women in 1381'),
 (16.0, None, None),
 (17.0, 'Pollard', 'Lord Fitzhugh’s Rising’'),
 (14.0, None, None),
 (13.0, 'B. Hanawalt', 'rime and Conflict in English Communities, 1300–1348'),
 (15.0, 'A.J. Pollard', 'Lord Fitzhugh’s Rising in 1470'),
 (18.0, 'Pollard', 'Lord Fitzhugh’s Rising’'),
 (12.0,
  'R.A. Griffiths',
  'The Reign of King Henry VI: The Exercise of Royal Authority, 1422–1461'),
 (19.0, None, None)]

In [19]:
#(113, 'M. Young and P. Willmott', 'Family and Kinship in East London'),
#('113', 'Young, M. and Willmott, P.', 'Family and Kinship in East London'),